# **Section 3: Form a portfolio, data selection, and necessary transformation**

In [13]:
import pandas as pd
import yfinance as yf
import numpy as np
from statsmodels.tsa.stattools import adfuller
from portfolio import *

## Part 1 - Creating portfolio

In [3]:
tickers = [
    'AAPL', 'MSFT',                                   # Information Technology
    'BIIB', 'JNJ', 'LLY', 'MRK', 'PFE',               # Health Care
    'AMZN', 'NKE',                                    # Consumer Discretionary
    'JPM', 'BAC', 'C', 'MS',                          # Financials
    'GOOGL',                                          # Communication Services
    'HON', 'UNP',                                     # Industrials
    'PG', 'KO', 'WMT', 'CL', 'TSN',                   # Consumer Staples
    'XOM', 'CVX',                                     # Energy
    'NEE',                                            # Utilities
    'PLD', 'AMT',                                     # Real Estate
    'LIN'                                             # Materials
]

equal_weight = 1/len(tickers)
weights = {ticker:equal_weight for ticker in tickers}
prices = yf.download(tickers, start='1976-04-01', end='2023-10-01', interval='1mo', progress=False)['Adj Close']

## Part 2 - Clean data

In [4]:
prices.index = pd.to_datetime(prices.index)
prices.index = prices.index.strftime('%Y-%m')
prices.dropna(inplace=True)
prices.head()

Ticker,AAPL,AMT,AMZN,BAC,BIIB,C,CL,CVX,GOOGL,HON,...,MSFT,NEE,NKE,PFE,PG,PLD,TSN,UNP,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2004-09,0.583704,11.720683,2.0430,27.297621,61.169998,278.599152,14.123345,25.307062,3.235232,21.670815,...,17.250334,4.690566,7.723265,12.824572,30.943264,18.852247,11.653690,9.766553,11.790461,24.749004
2004-10,0.789319,13.125636,1.7065,28.502569,58.160000,280.177856,13.948284,25.033421,4.758987,20.353401,...,17.449980,4.730387,7.979585,12.133054,29.262318,19.096680,10.547972,10.549776,11.950030,25.204758
2004-11,1.009997,13.843387,1.9840,29.444391,58.680000,285.172699,14.453038,25.759979,4.542805,21.350527,...,16.726267,4.828566,8.308353,11.638513,30.722250,20.580601,11.922843,10.628515,11.537802,26.244291
2004-12,0.970079,14.049550,2.2145,29.902548,66.610001,307.030731,16.077797,24.962437,4.812658,21.512030,...,18.579523,5.180114,8.900119,11.337539,31.641399,20.807276,13.417186,11.266831,11.706240,26.391447
2005-01,1.158371,13.835752,2.1610,29.797808,64.959999,312.574768,16.511477,25.860916,4.883303,21.858313,...,18.273573,5.311088,8.513958,10.186497,30.578648,19.383286,12.520276,10.032343,11.641442,26.566486


## Part 4 - Stationarity tests on MEVs

In [5]:
MEVs = pd.read_csv('2024-Table_2A_Historic_Domestic.csv')
quarter_to_month = {'Q1': '03', 'Q2': '06', 'Q3': '09', 'Q4': '12'}

def convert_to_yyyy_mm(quarter_str):
    year, quarter = quarter_str.split()
    month = quarter_to_month[quarter]
    return f"{year}-{month}"
    
MEVs['Date'] = MEVs['Date'].apply(convert_to_yyyy_mm)
MEVs.set_index('Date', inplace=True)
MEVs.drop(columns=['Scenario Name'], inplace=True)

MEVs.head()

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level)
Date,,,,,,,,,,,,,,,,
1976-03,9.3,14.0,5.0,9.6,7.7,4.7,4.9,7.4,7.6,NaN,8.9,6.8,NaN,22.9,50.9,NaN
1976-06,3.0,7.2,2.3,5.8,7.6,3.6,5.2,7.4,7.6,NaN,8.8,6.9,NaN,23.6,51.8,NaN
1976-09,2.2,7.6,3.2,9.6,7.7,6.5,5.2,7.3,7.6,NaN,9.0,7.1,NaN,24.2,52.6,NaN
1976-12,2.9,10.5,2.6,9.2,7.8,5.9,4.7,6.5,7.1,NaN,8.8,6.5,NaN,25.2,53.4,NaN
1977-03,4.8,11.7,0.9,8.4,7.5,7.5,4.6,6.8,7.2,NaN,8.7,6.3,NaN,26.2,55.0,NaN


In [6]:
def make_stationary(series, significance_level=0.05):
    """
    Differentiates a pandas Series until it becomes stationary based on the Augmented Dickey-Fuller test.
    
    Parameters:
    - series: pd.Series - The time series data to test for stationarity.
    - significance_level: float - The p-value threshold to consider the series stationary (default is 0.05).
    
    Returns:
    - num_diffs: int - The number of differences needed to achieve stationarity.
    """
    num_diffs = 0
    diff_series = series.copy()
    
    while True:

        adf_test = adfuller(diff_series.dropna())
        p_value = adf_test[1]
        
        if p_value < significance_level:
            return num_diffs, diff_series
        
        diff_series = diff_series.diff().dropna()
        num_diffs += 1

In [7]:
diffs_needed = pd.DataFrame(index=MEVs.columns, columns=['Differences'])
for MEV in MEVs.columns:
    stationary = make_stationary(MEVs[MEV])
    diffs_needed.loc[MEV] = stationary[0]
    if stationary[0] != 0:
        MEVs[MEV] = stationary[1]
MEVs = MEVs.dropna()
MEVs.head()

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level)
Date,,,,,,,,,,,,,,,,
1990-03,4.4,9.0,3.3,2.6,5.3,3.0,0.1,0.5,0.5,0.6,0.3,-0.5,-146.4,1.1,-1.0,27.3
1990-06,1.5,6.1,3.0,-2.6,5.3,-3.1,-0.1,0.2,0.3,0.3,0.2,0.0,150.9,0.0,-0.9,24.2
1990-09,0.3,3.7,0.1,-1.6,5.7,3.1,-0.2,-0.2,0.0,-0.1,-0.2,0.0,-545.1,-0.1,-0.5,36.5
1990-12,-3.6,-0.7,-3.2,-3.1,6.1,-0.1,-0.5,-0.4,-0.3,0.3,-0.1,0.0,222.1,-0.4,-0.4,34.0
1991-03,-1.9,2.0,1.2,1.3,6.6,-4.0,-1.0,-0.4,-0.3,-0.5,-0.5,-0.8,482.3,-0.6,-1.0,36.2


In [8]:
diffs_needed[diffs_needed['Differences']!=0]

,Differences
Nominal disposable income growth,1
CPI inflation rate,1
3-month Treasury rate,1
5-year Treasury yield,1
10-year Treasury yield,1
BBB corporate yield,1
Mortgage rate,1
Prime rate,1
Dow Jones Total Stock Market Index (Level),1
House Price Index (Level),1


## Part 5 - Summary Statistics

### For the stock returns:

In [9]:
prices.describe().T

,count,mean,std,min,25%,50%,75%,max
Ticker,,,,,,,,
AAPL,229.0,41.089456,51.693560,0.583704,5.583982,19.840931,45.121765,194.971771
AMT,229.0,93.712152,73.646325,11.720683,32.260506,65.349602,142.240707,267.634766
AMZN,229.0,45.629840,53.076733,1.344500,4.528000,16.122000,85.936501,175.353500
BAC,229.0,21.329719,10.299661,3.136746,12.113324,21.555002,29.444391,44.192055
BIIB,229.0,188.913712,115.398784,34.450001,57.389999,219.289993,289.730011,422.239990
C,229.0,94.389897,107.034425,11.438826,35.938503,43.804787,61.727081,384.071869
CL,229.0,45.405716,19.576115,13.948284,26.109671,50.561008,61.057644,79.470909
CVX,229.0,70.391147,32.970923,24.891180,43.791649,69.687386,85.466713,169.508759
GOOGL,229.0,40.853445,37.465174,3.235232,12.862295,26.935305,56.320042,147.680328


### The stock prices range from October of 2004 up to October of 2023 on a monthly basis.

### For the MEVs:

In [10]:
MEVs.describe().T

,count,mean,std,min,25%,50%,75%,max
Real GDP growth,136.0,2.541912,4.532672,-28.0,1.475,2.55,4.025,34.8
Nominal GDP growth,136.0,4.873529,5.098456,-29.2,3.600,5.00,6.500,39.7
Real disposable income growth,136.0,2.811029,7.538795,-27.6,1.200,2.75,4.225,56.0
Nominal disposable income growth,136.0,-0.017647,12.432013,-86.6,-1.925,0.20,2.000,69.3
Unemployment rate,136.0,5.758088,1.742946,3.5,4.475,5.40,6.700,13.0
CPI inflation rate,136.0,-0.009559,2.423462,-15.2,-1.200,0.00,1.025,8.4
3-month Treasury rate,136.0,-0.017647,0.426334,-1.3,-0.100,0.00,0.200,1.6
5-year Treasury yield,136.0,-0.025735,0.415977,-1.1,-0.300,0.00,0.200,1.2
10-year Treasury yield,136.0,-0.025735,0.368178,-0.9,-0.300,0.00,0.200,1.0
BBB corporate yield,136.0,-0.026471,0.484613,-1.6,-0.300,0.00,0.225,2.6


### The MEVs range from Q1 of 1990 to Q3 of 2023. Chances are in the data analysis where the MEVs are used, the years that are in this dataset preceding those in the stock returns dataset will be dropped.

## **Section 4: Stress Testing using Fama-French three-factor model**

## Part 1 - Picking a subset of MEVs

### We're going to try and capture the big economic picture at each time step while avoiding redundancy. In order to do this, we chose to use the CPI, real GDP growth, real disposable income growth, 3 month, 5 year, and 10 year rates.

In [ ]:
MEVs = MEVs[['Real GDP growth', 
             'Real disposable income growth', 
             'CPI inflation rate', 
             '3-month Treasury rate',
             '5-year Treasury yield',
             '10-year Treasury yield',
             'Market Volatility Index (Level)'
            ]]

## Part 2 - Report the results from the Fama-French three factor model

In [12]:
#portfolio returns

returns = prices.pct_change()[1:]
returns['portfolio'] = sum(returns[stock] * weights[stock] for stock, weight in weights.items()) # Setting the portfolio returns based on weights

In [30]:
FF_factors = pd.read_excel('wrds_data.xlsx', sheet_name='returns', index_col=0)

c:\Users\justi\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [63]:
FF_factors = pd.read_excel('wrds_data.xlsx', sheet_name='returns', index_col=0)
FF_factors = FF_factors[['FAMA-FRENCH MARKET FACTOR', 'FAMA-FRENCH SIZE FACTOR (SMB)', 'FAMA-FRENCH VALUE FACTOR (HML)', 'MOMENTUM FACTOR']]
FF_factors = FF_factors.rename(columns={'FAMA-FRENCH MARKET FACTOR': 'MKT', 'FAMA-FRENCH SIZE FACTOR (SMB)': 'SMB', 'FAMA-FRENCH VALUE FACTOR (HML)': 'HML', 'MOMENTUM FACTOR': 'UMD'})
FF_factors.head()


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,MKT,RF,SMB,HML,UMD
Date,,,,,
1963-01,0.0493,0.0025,0.0308,0.0221,-0.0211
1963-02,-0.0238,0.0023,0.0048,0.0218,0.0253
1963-03,0.0308,0.0023,-0.0259,0.0206,0.0162
1963-04,0.0451,0.0025,-0.0134,0.0100,-0.0009
1963-05,0.0176,0.0024,0.0113,0.0254,0.0033


In [64]:
FF_model = FF_factors.copy()
FF_model['portfolio'] = returns['portfolio']
FF_model = FF_model.dropna()
FF_model

,MKT,SMB,HML,UMD,portfolio
Date,,,,,
2004-10,0.0143,0.0015,-0.0022,-0.0138,0.022147
2004-11,0.0454,0.0374,0.0141,0.0316,0.029243
2004-12,0.0343,-0.0003,-0.0022,-0.0287,0.051720
2005-01,-0.0276,-0.0172,0.0206,0.0305,-0.013803
2005-02,0.0189,-0.0057,0.0154,0.0337,0.014125
...,...,...,...,...,...
2023-05,0.0035,0.0160,-0.0774,-0.0065,-0.034575
2023-06,0.0647,0.0155,-0.0020,-0.0235,0.043780
2023-07,0.0321,0.0205,0.0411,-0.0405,0.022336


In [ ]:
FF_portfolio = get_ols_metrics(FF_model[['MKT', 'SMB', 'HML', 'UMD']], FF_model['portfolio'])
FF_portfolio

,alpha,MKT,SMB,HML,UMD,r-squared,Info Ratio
portfolio,0.005963,0.912181,-0.271542,0.096881,-0.017913,0.901248,0.448041


## The Fama-French model for our portfolio is given by: 
$$
\Huge
E[r_i] = 0.92(MKT - r_f) - 0.3(SMB) + 0.1(HML) - 0.02(UMD) + 0.01 
$$

## Part 3 - Identify the impact of our chosen MEVs on Fama-French factors.

### Note: Here we merged the dataframes of the FF factors with the MEVs dataframe, inherently dropping all months that don't fall at the end of a quarter. This was to maintain uniformity and ensure an accurate result. We felt this method was better than interpolating MEV data because those numbers come straight from the Fed and it wouldn't be reliable to try and subjectively interpret gaps in data.

In [ ]:
FF_model_MEVs = FF_factors.merge(MEVs, left_index=True, right_index=True, how='inner')

In [ ]:
alltime_MEVs_effects_on_factors = pd.DataFrame(index=FF_model.columns[:-1], columns=MEVs.columns)
regressors = FF_model_MEVs[MEVs.columns]

In [ ]:
for factor in FF_model.columns[:-1]:
    reg = get_ols_metrics(regressors, FF_model_MEVs[factor])
    for MEV in reg.columns[1:-2]:
        alltime_MEVs_effects_on_factors.loc[factor, MEV] = reg[MEV].values[0]

In [ ]:
alltime_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
MKT,0.001038,0.000347,-0.000258,0.000558,-0.011286,-0.000289
SMB,0.000407,-0.000307,-0.000689,0.000178,-0.031251,0.031509
HML,-0.000108,0.000581,0.000561,-0.00137,0.011316,-0.011033
UMD,0.001034,-0.000257,-0.000657,-0.006656,-0.000665,-0.0067


### This dataframe shows the coefficients of the linear regressions:
$$
\Huge
FF_i = \alpha_i + \beta_{1_i}MEV_1 + \beta_2MEV_2 + \ldots + \beta_nMEV_n
$$

## Part 4 - Investigating the impact of the MEVs on Fama-French factors during *stressed times*

### We are picking all date ranges of stressed times after 1976, where our data for FF factors and MEVs starts. This gives a full picture on the effect of the MEVs on the FF factors throughout history of the past ~5 decades, whereas a smaller sample size wouldn't accurately describe the effects.

In [ ]:
df = FF_factors.merge(MEVs, left_index=True, right_index=True, how='inner')
stressed_date_ranges = [('1980-01', '1980-06'),
                        ('1981-06', '1982-12'),
                        ('1990-06', '1991-03'),
                        ('2001-03', '2001-12'),
                        ('2007-12', '2009-06'),
                        ('2020-03', '2020-06')
                       ]
stressed_data = pd.concat(
    [df.loc[start:end] for start, end in stressed_date_ranges]
)

In [ ]:
stressed_MEVs_effects_on_factors = pd.DataFrame(index=FF_model.columns[:-1], columns=MEVs.columns)
regressors = stressed_data[MEVs.columns]
for factor in FF_model.columns[:-1]:
    reg = get_ols_metrics(regressors, stressed_data[factor])
    for MEV in reg.columns[1:-2]:
        stressed_MEVs_effects_on_factors.loc[factor, MEV] = reg[MEV].values[0]
stressed_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
MKT,-0.003533,-0.001477,0.000422,-0.005826,0.021461,-0.043141
SMB,-0.000399,-0.000226,-0.001747,-0.016149,0.017437,-0.008091
HML,0.000063,-0.000588,0.000285,0.00031,0.007701,-0.002738
UMD,0.006836,0.003736,0.001189,-0.007574,-0.022251,0.003064


### This table similarly shows the coefficients of the MEVs on FF factors.

## Part 5 - Projecting the performance of our portfolio

In [ ]:
# Read in the adverse situation MEVs
MEVs_severe = pd.read_csv('MEV_severe.csv')
MEVs_severe['Date'] = MEVs_severe['Date'].apply(convert_to_yyyy_mm)
MEVs_severe.set_index('Date', inplace=True)
MEVs_severe = MEVs_severe[MEVs.columns] # Filter to only the MEVs we're using

In [ ]:
# Handle stationarity similar to before
diffs_needed = pd.DataFrame(index=MEVs_severe.columns, columns=['Differences'])
for MEV in MEVs_severe.columns:
    stationary = make_stationary(MEVs_severe[MEV])
    diffs_needed.loc[MEV] = stationary[0]
    if stationary[0] != 0:
        MEVs_severe[MEV] = stationary[1]
MEVs_severe = MEVs_severe.dropna()

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:958: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
Date,,,,,,
1976-06,3.0,2.3,-1.1,0.3,0.0,0.0
1976-09,2.2,3.2,2.9,0.0,-0.1,0.0
1976-12,2.9,2.6,-0.6,-0.5,-0.8,-0.5
1977-03,4.8,0.9,1.6,-0.1,0.3,0.1
1977-06,8.0,3.8,-0.3,0.2,0.0,0.1


In [ ]:
MEVs_severe.head()

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
Date,,,,,,
2025-03,2.0,2.8,0.1,0.1,0.0,4.000000e-01
2025-06,-1.7,1.3,0.0,0.1,0.1,-2.000000e-01
2025-09,-2.1,0.5,0.0,0.1,0.1,1.000000e-01
2025-12,5.3,3.6,0.1,0.1,0.0,-4.440892e-16
2026-03,-6.0,0.3,0.0,0.1,0.1,7.771561e-16


In [ ]:
alltime_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield
MKT,0.001038,0.000347,-0.000258,0.000558,-0.011286,-0.000289
SMB,0.000407,-0.000307,-0.000689,0.000178,-0.031251,0.031509
HML,-0.000108,0.000581,0.000561,-0.00137,0.011316,-0.011033
UMD,0.001034,-0.000257,-0.000657,-0.006656,-0.000665,-0.0067


In [ ]:
# TODO: run the regression using all time MEV coefficients against severe MEV values to get MKT, SMB, HML, UMD values 
# and then plug those values into the regression equation for earlier at each time step to get portfolio return